In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'  # or any {'0', '1', '2'}
from imagenet_dataset import get_dataset
from retina_env import RetinaEnv, calculate_retinal_filter
from rl_networks import create_actor_model, create_critic_model, policy
from rl_core import Buffer, update_target
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import types
config = types.SimpleNamespace()
config.batch_size = 32
config.margin = 20
config.image_h = 224
config.image_w = 224
config.image_hm = config.image_h+2*config.margin
config.image_wm = config.image_w+2*config.margin
config.foveate = None
config.do_grayscale = True
config.history_length = 16
config.t_ignore = 16
config.t_max =50
config.motion_mode = 'velocity'
config.use_dones = True

config.gym_mode = False
t_vec = np.linspace(0,150,16)

balanced_filter = calculate_retinal_filter(t_vec, R=1.0)
config.filter = balanced_filter.reshape([1,1,-1,1])
config.min_freq = 1
config.max_freq = 13
config.action_upper_bound = np.array([2.0, 2.0])

dataset_dir = '/home/bnapp/datasets/tensorflow_datasets/imagenet2012/5.0.0/'

def epsilon_scheduler(episode, floor_episode=200, epsilon_floor=0.1):
    if episode < floor_episode:
        return 1.-(1.-epsilon_floor)*episode/floor_episode
    else:
        return epsilon_floor



env =  BatchMountainCarContinuous(batch_size=config.batch_size)


if config.gym_mode:
    num_states = env.observation_space.shape[0]
    num_actions = env.action_space.shape[0]

    upper_bound = env.action_space.high[0]
    lower_bound = env.action_space.low[0]
else:
    num_states = env.observation_size
    num_actions = env.action_size
    upper_bound = env.action_upper_bound
    lower_bound = env.action_lower_bound

# You might want to adjust the hyperparameters
actor_lr = 0.0001
critic_lr = 0.001
gamma = 0.99
tau = 0.005

buffer_capacity = 10000



# Create actor and critic networks
actor_model = create_actor_model(env.image_h, env.image_w, env.spectral_density_size, env.location_history_size, env.timestep_size, env.action_size)
critic_model = create_critic_model(env.image_h, env.image_w, env.spectral_density_size, env.location_history_size, env.timestep_size, env.action_size)

# Create target actor and critic networks
target_actor = create_actor_model(env.image_h, env.image_w, env.spectral_density_size, env.location_history_size, env.timestep_size, env.action_size)
target_critic = create_critic_model(env.image_h, env.image_w, env.spectral_density_size, env.location_history_size, env.timestep_size, env.action_size)

# Making the weights equal initially
target_actor.set_weights(actor_model.get_weights())
target_critic.set_weights(critic_model.get_weights())

# Learning rate for actor-critic models
critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

# Experience replay buffer
buffer = Buffer(buffer_capacity, config.batch_size, num_states=num_states, num_actions=num_actions,
                state_reshape_fn=env.unflatten_observation, use_dones=config.use_dones)

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
# Training loop
reward_records = []
epsilon_records = []
action_mean_records = []
action_var_records = []
episodes = 10000
for ep in range(episodes):
    prev_state = env.reset(images=images)
    episodic_reward = 0
    epsilon = epsilon_scheduler(ep, floor_episode=1000)

    while True:
        # tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)
        
        deterministic_action = policy(env.unflatten_observation(prev_state), actor_model, lower_bound, upper_bound)
        random_action = -2+4*np.random.uniform(size=(config.batch_size,2))

        if env.warmup_done:
            action = epsilon*random_action + (1-epsilon)*deterministic_action
        else:
            action = random_action

        # Recieve state and reward from environment
        state, reward, done, info = env.step(10*action)
        
        if config.use_dones:
            buffer.record((prev_state, action, reward, state, done))
        else:   
            buffer.record((prev_state, action, reward, state))

        episodic_reward += reward

        buffer.learn(actor_model, target_actor, critic_model, target_critic, actor_optimizer, critic_optimizer, gamma, tau)
        update_target(target_actor.variables, actor_model.variables, tau)
        update_target(target_critic.variables, critic_model.variables, tau)

        action_mean_records.append(deterministic_action.numpy().mean(axis=0))
        action_var_records.append(deterministic_action.numpy().var(axis=0))
        
        if done:
            break

        #prev_state = state avoid assingment by reference:
        prev_state = np.copy(state)
#         print('debug action: ', action[0])

    print(f"Episode * {ep} * exploration epsilon {epsilon} * Episodic Reward is ==> {episodic_reward.numpy().mean()}")
    print("action means and variances at step -10:", action_mean_records[-10],action_var_records[-10])
    print("action means and variances at step -5:", action_mean_records[-5],action_var_records[-5])
    reward_records.append(episodic_reward.numpy().mean())
    epsilon_records.append(epsilon)

Episode * 0 * exploration epsilon 1.0 * Episodic Reward is ==> -12.085285186767578
action means and variances at step -10: [ 0.1957888 -0.4297591] [0.00220437 0.00051978]
action means and variances at step -5: [ 0.18302326 -0.5028217 ] [0.00221128 0.0005898 ]
Episode * 1 * exploration epsilon 0.9991 * Episodic Reward is ==> -11.289146423339844
action means and variances at step -10: [-0.3747945 -0.5470153] [0.00062857 0.00033525]
action means and variances at step -5: [-0.47265172 -0.52397805] [0.00052581 0.00020283]
Episode * 2 * exploration epsilon 0.9982 * Episodic Reward is ==> -11.516767501831055
action means and variances at step -10: [-0.59991205 -0.74750835] [0.00025864 0.00034701]
action means and variances at step -5: [-0.63800186 -0.76517045] [0.0002392 0.0003766]
Episode * 3 * exploration epsilon 0.9973 * Episodic Reward is ==> -12.017725944519043
action means and variances at step -10: [-0.7687416 -0.9391137] [0.00036351 0.00055186]
action means and variances at step -5: [

Episode * 16 * exploration epsilon 0.9856 * Episodic Reward is ==> -11.727266311645508
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 17 * exploration epsilon 0.9847 * Episodic Reward is ==> -11.993698120117188
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 18 * exploration epsilon 0.9838 * Episodic Reward is ==> -11.47765827178955
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 19 * exploration epsilon 0.9829 * Episodic Reward is ==> -11.764754295349121
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 20 * exploration epsilon 0.982 * Episodic Reward is ==> -11.477373123168945
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] 

Episode * 34 * exploration epsilon 0.9694 * Episodic Reward is ==> -12.105091094970703
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 35 * exploration epsilon 0.9685 * Episodic Reward is ==> -11.543885231018066
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 36 * exploration epsilon 0.9676 * Episodic Reward is ==> -12.195247650146484
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 37 * exploration epsilon 0.9667 * Episodic Reward is ==> -11.67061710357666
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 38 * exploration epsilon 0.9658 * Episodic Reward is ==> -11.701221466064453
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-1.91844

Episode * 52 * exploration epsilon 0.9532 * Episodic Reward is ==> -11.700263977050781
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 53 * exploration epsilon 0.9523 * Episodic Reward is ==> -11.548986434936523
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 54 * exploration epsilon 0.9514 * Episodic Reward is ==> -11.211820602416992
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 55 * exploration epsilon 0.9505 * Episodic Reward is ==> -11.544827461242676
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 56 * exploration epsilon 0.9496 * Episodic Reward is ==> -11.676017761230469
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.

Episode * 70 * exploration epsilon 0.937 * Episodic Reward is ==> -11.715580940246582
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 71 * exploration epsilon 0.9361 * Episodic Reward is ==> -11.844799041748047
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 72 * exploration epsilon 0.9352 * Episodic Reward is ==> -11.811332702636719
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 73 * exploration epsilon 0.9343 * Episodic Reward is ==> -11.816896438598633
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.] [0. 0.]
Episode * 74 * exploration epsilon 0.9334 * Episodic Reward is ==> -11.015264511108398
action means and variances at step -10: [-2. -2.] [0. 0.]
action means and variances at step -5: [-2. -2.]

...

Episode * 5010 * exploration epsilon 0.1 * Episodic Reward is ==> -15.064154624938965
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5011 * exploration epsilon 0.1 * Episodic Reward is ==> -15.274532318115234
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5012 * exploration epsilon 0.1 * Episodic Reward is ==> -15.625336647033691
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5013 * exploration epsilon 0.1 * Episodic Reward is ==> -15.4870023727417
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5014 * exploration epsilon 0.1 * Episodic Reward is ==> -15.489986419677734
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0

Episode * 5028 * exploration epsilon 0.1 * Episodic Reward is ==> -15.141779899597168
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5029 * exploration epsilon 0.1 * Episodic Reward is ==> -15.436552047729492
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5030 * exploration epsilon 0.1 * Episodic Reward is ==> -15.444096565246582
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5031 * exploration epsilon 0.1 * Episodic Reward is ==> -15.470304489135742
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5032 * exploration epsilon 0.1 * Episodic Reward is ==> -15.537304878234863
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0.

Episode * 5046 * exploration epsilon 0.1 * Episodic Reward is ==> -15.645345687866211
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5047 * exploration epsilon 0.1 * Episodic Reward is ==> -15.806713104248047
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5048 * exploration epsilon 0.1 * Episodic Reward is ==> -15.442919731140137
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5049 * exploration epsilon 0.1 * Episodic Reward is ==> -15.754854202270508
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5050 * exploration epsilon 0.1 * Episodic Reward is ==> -15.418018341064453
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0.

Episode * 5064 * exploration epsilon 0.1 * Episodic Reward is ==> -15.431245803833008
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5065 * exploration epsilon 0.1 * Episodic Reward is ==> -15.764019012451172
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5066 * exploration epsilon 0.1 * Episodic Reward is ==> -15.187171936035156
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5067 * exploration epsilon 0.1 * Episodic Reward is ==> -15.424558639526367
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 0.]
Episode * 5068 * exploration epsilon 0.1 * Episodic Reward is ==> -15.65575122833252
action means and variances at step -10: [ 2. -2.] [0. 0.]
action means and variances at step -5: [ 2. -2.] [0. 

KeyboardInterrupt: 

In [ ]:
plt.plot(reward_records)

In [ ]:
import misc

In [ ]:
plt.plot(reward_records)
plt.plot(misc.smooth(reward_records,100))
plt.grid()


In [ ]:
plt.plot(reward_records)
plt.plot(misc.smooth(reward_records,1000))
plt.grid()

In [ ]:
plt.plot(buffer.critic_loss_buffer[100:])
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],100))

In [ ]:
plt.plot(buffer.critic_loss_buffer[:10])


In [ ]:
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],1))
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],100))
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],1000))
plt.ylim([0,0.05])
plt.grid()

In [ ]:
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],1))
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],100))
plt.plot(misc.smooth(buffer.critic_loss_buffer[100:],1000))
plt.grid()

In [ ]:
plt.plot(misc.smooth(buffer.critic_loss_buffer[-500:],1))
plt.plot(misc.smooth(buffer.critic_loss_buffer[-500:],100))
plt.plot(misc.smooth(buffer.critic_loss_buffer[-500:],1000))
plt.grid()

In [ ]:
## Debugging Nan values in action

In [ ]:
plt.plot(buffer.actor_loss_buffer[100:])


In [ ]:
deterministic_action.numpy()

In [ ]:
reward

In [ ]:
env.reset()
for ii in range(55):
    random_action = -2+4*np.random.uniform(size=(config.batch_size,2))
    state, reward, done, info = env.step(random_action)
    print('----------')
    print(reward.numpy()[:5])
    print(critic_model([env.unflatten_observation(state),
                          deterministic_action]).numpy()[:5].T)
    print(critic_model([env.unflatten_observation(state),
                          random_action]).numpy()[:5].T)

In [ ]:
plt.imshow(env.unflatten_observation(state)[0].numpy()[0])

In [ ]:
env.unflatten_observation(state)[1].numpy()[0]

In [ ]:
env.unflatten_observation(state)[2].numpy()[0]

In [ ]:
env.unflatten_observation(state)[3].numpy()[0]

In [ ]:
random_action

In [ ]:
critic_model([env.unflatten_observation(state),
                          random_action])

In [ ]:
deterministic_action

In [ ]:
actor_model(env.unflatten_observation(state))

In [ ]:
actor_model.weights

In [ ]:
critic_model.weights